In [ ]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

# Configuration
n_games = 1
interactive_mode = False
n_players = 8  # Number of players 6 currently works well.
n_werewolves = 2  # Number of werewolves
n_seers = 1  # Number of seers

# Read names from names.txt
with open('names.txt', 'r') as file:
    all_names = [line.strip() for line in file.readlines()]

# Ensure there are enough names
if len(all_names) < n_players:
    raise ValueError("Not enough names in names.txt to assign to all players")

# Randomly select player names
np.random.shuffle(all_names)
player_names = all_names[:n_players]
all_player_names = player_names.copy()  # Keep a copy of all player names

# Assign roles
roles = [1] * n_werewolves + [2] * n_seers + [0] * (n_players - n_werewolves - n_seers)
np.random.shuffle(roles)

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    while not stop:
        # Debate stage
        while rounds < 15 and not stop:
            rounds += 1
            next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

            if next_to_speak != "vote":
                # If next to speak is first player and interactive mode, then ask for input
                if next_to_speak == player_names[0].lower() and interactive_mode:
                    print("Your turn to speak")
                    uncensored = "Thomas: " + input()
                    censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                else:
                    censored, uncensored = players[next_to_speak].get_player_text()
                
                for name in player_names:
                    if name.lower() != next_to_speak:
                        players[name.lower()].add_other_text(censored)
            
                conversation_history.append(censored)
                conversation_history_uncensored.append(uncensored)

                if interactive_mode:
                    print(censored)
                else:
                    print(uncensored)
            else:
                break

        # Seer stage
        seer_discoveries = []
        for seer_name in seer_names:
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

            chosen_role = players[chosen_player].role
            players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            seer_discoveries.append((seer_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce seer's discovery
        for seer_name, chosen_player, chosen_role in seer_discoveries:
            announcement = f"{seer_name.capitalize()} announces: I have discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            if interactive_mode:
                print(announcement)

        # Voting stage
        votes = {name.lower(): 0 for name in player_names}

        vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
        conversation_history.append(vote_prompt.content)
        conversation_history_uncensored.append(vote_prompt.content)
        print(vote_prompt.content)

        for name in player_names:
            players[name.lower()].add_other_text(vote_prompt.content)

        for voting_name in player_names:
            if voting_name.lower() in werewolf_names:
                players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolves are (including you): {', '.join(werewolf_names)}. If a wolf dies, then you also lose!]\n"

            if voting_name.lower() == player_names[0].lower() and interactive_mode:
                print("Your turn to vote")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[voting_name.lower()].get_player_text()

            for name in player_names:
                if name.lower() != voting_name:
                    players[name.lower()].add_other_text(censored)
            
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            voted_for = ""
            search = censored.lower()
            for name in player_names:
                if name.lower() in search:
                    voted_for = name.lower()
                    search = search.split(name.lower())[-1]

            try:
                votes[voted_for] += 1
            except KeyError:
                pass
            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        print(votes)

        # Determine who is voted out
        voted_out = max(votes, key=votes.get)
        player_names.remove(voted_out.capitalize())
        del players[voted_out]

        # Check win conditions
        if len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            break
        elif not werewolf_names:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            break

        # Werewolf elimination stage
        if werewolf_names and not stop:
            if interactive_mode:
                print("Werewolves, choose a player to eliminate.")
                chosen_victim = input("Enter the name of the player you want to eliminate: ").strip().lower()
            else:
                chosen_victim = np.random.choice([name.lower() for name in player_names if name.lower() not in werewolf_names])

            announcement = f"The werewolves have chosen to eliminate {chosen_victim.capitalize()}."
            for name in player_names:
                players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            print(announcement)

            player_names.remove(chosen_victim.capitalize())
            del players[chosen_victim]

            if len(werewolf_names) >= len(player_names) - len(werewolf_names):
                stop = True
                winner = "werewolves"
                print("The werewolves have won!")
            elif not werewolf_names:
                stop = True
                winner = "villagers"
                print("The villagers have won!")

        # Reset rounds for the next debate stage
        rounds = 0

    # Save the game state
    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,
        "all_player_names": all_player_names,  # Add all player names to the JSON dump
        "remaining_player_names": player_names,  # Add remaining player names to the JSON dump
        "model": model,
        "interactive": interactive_mode,
    }, open(f"../data/full_games/final_project_no_custom{game}.json", "w+"), indent=4) # correct path is ../data/full_games/final_project_no_custom{game}.json

  0%|          | 0/1 [00:00<?, ?it/s]

Tina: I suggest we start by sharing our suspicions. Bob's death weighs heavy on us, and we must act swiftly. [I need to guide the conversation towards finding the Werewolf without revealing my Seer role.] Let's hear everyone's thoughts before we make a decision. [I must appear calm and collected, but also assertive in my approach to identifying the Werewolf.]
Rachel: [Rachel is internally pleased with the chaos the Werewolves have caused. It's time to sow more discord among the villagers.] I find it quite suspicious that Tina is taking the lead so quickly. It's as if she's trying to divert attention from herself. Let's not forget that the Werewolves are cunning and might be hiding in plain sight.
Victor: [Rachel's accusation against Tina seems valid. Her eagerness to take charge does raise some suspicion. I should keep an eye on both of them.] I agree with Rachel, we should be cautious and not rush into accusations. Let's observe closely before we decide our next move.
Hannah: [Observi

  0%|          | 0/1 [00:36<?, ?it/s]


KeyError: 'tina'

In [38]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

# Configuration
n_games = 1
interactive_mode = False
n_players = 8  # Number of players
n_werewolves = 2  # Number of werewolves
n_seers = 1  # Number of seers

# Read names from names.txt
with open('names.txt', 'r') as file:
    all_names = [line.strip().lower() for line in file.readlines()]

# Ensure there are enough names
if len(all_names) < n_players:
    raise ValueError("Not enough names in names.txt to assign to all players")

# Randomly select player names
np.random.shuffle(all_names)
player_names = all_names[:n_players]
all_player_names = player_names.copy()  # Keep a copy of all player names

# Assign roles
roles = [1] * n_werewolves + [2] * n_seers + [0] * (n_players - n_werewolves - n_seers)
np.random.shuffle(roles)

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    while not stop:
        # Debate stage
        while rounds < 15 and not stop:
            rounds += 1
            next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

            if next_to_speak != "vote":
                # If next to speak is first player and interactive mode, then ask for input
                if next_to_speak == player_names[0].lower() and interactive_mode:
                    print("Your turn to speak")
                    uncensored = "Thomas: " + input()
                    censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                else:
                    censored, uncensored = players[next_to_speak].get_player_text()
                
                for name in player_names:
                    if name.lower() != next_to_speak:
                        players[name.lower()].add_other_text(censored)
            
                conversation_history.append(censored)
                conversation_history_uncensored.append(uncensored)

                if interactive_mode:
                    print(censored)
                else:
                    print(uncensored)
            else:
                break

        # Seer stage
        seer_discoveries = []
        for seer_name in seer_names:
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

            chosen_role = players[chosen_player].role
            players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            seer_discoveries.append((seer_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce seer's discovery
        for seer_name, chosen_player, chosen_role in seer_discoveries:
            announcement = f"{seer_name.capitalize()} announces: I have discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            if interactive_mode:
                print(announcement)

        # Voting stage
        votes = {name.lower(): 0 for name in player_names}

        vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
        conversation_history.append(vote_prompt.content)
        conversation_history_uncensored.append(vote_prompt.content)
        print(vote_prompt.content)

        for name in player_names:
            players[name.lower()].add_other_text(vote_prompt.content)

        for voting_name in player_names:
            if voting_name.lower() in werewolf_names:
                players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolves are (including you): {', '.join(werewolf_names)}. If a wolf dies, then you also lose!]\n"

            if voting_name.lower() == player_names[0].lower() and interactive_mode:
                print("Your turn to vote")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[voting_name.lower()].get_player_text()

            for name in player_names:
                if name.lower() != voting_name:
                    players[name.lower()].add_other_text(censored)
            
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            voted_for = ""
            search = censored.lower()
            for name in player_names:
                if name.lower() in search:
                    voted_for = name.lower()
                    search = search.split(name.lower())[-1]

            try:
                votes[voted_for] += 1
            except KeyError:
                print(f"Warning: {voted_for} not found in votes dictionary")
            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        print(votes)

        # Determine who is voted out
        voted_out = max(votes, key=votes.get)
        player_names.remove(voted_out)
        del players[voted_out]

        # Check win conditions
        if len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            break
        elif not werewolf_names:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            break

        # Werewolf elimination stage
        if werewolf_names and not stop:
            if interactive_mode:
                print("Werewolves, choose a player to eliminate.")
                chosen_victim = input("Enter the name of the player you want to eliminate: ").strip().lower()
            else:
                chosen_victim = np.random.choice([name.lower() for name in player_names if name.lower() not in werewolf_names])

            announcement = f"The werewolves have chosen to eliminate {chosen_victim.capitalize()}."
            for name in player_names:
                players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            print(announcement)

            player_names.remove(chosen_victim)
            del players[chosen_victim]

            if len(werewolf_names) >= len(player_names) - len(werewolf_names):
                stop = True
                winner = "werewolves"
                print("The werewolves have won!")
            elif not werewolf_names:
                stop = True
                winner = "villagers"
                print("The villagers have won!")

            if not stop:
                # Announce the elimination and allow for additional discussion
                additional_discussion_prompt = f"The werewolves have eliminated {chosen_victim.capitalize()} during the night. You have some time for additional discussion before the next voting phase."
                conversation_history.append(additional_discussion_prompt)
                conversation_history_uncensored.append(additional_discussion_prompt)
                print(additional_discussion_prompt)

                for name in player_names:
                    players[name.lower()].add_other_text(additional_discussion_prompt)

                # Additional discussion stage
                while rounds < 15 and not stop:
                    rounds += 1
                    next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

                    if next_to_speak != "vote":
                        # If next to speak is first player and interactive mode, then ask for input
                        if next_to_speak == player_names[0].lower() and interactive_mode:
                            print("Your turn to speak")
                            uncensored = "Thomas: " + input()
                            censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                        else:
                            censored, uncensored = players[next_to_speak].get_player_text()
                        
                        for name in player_names:
                            if name.lower() != next_to_speak:
                                players[name.lower()].add_other_text(censored)
                    
                        conversation_history.append(censored)
                        conversation_history_uncensored.append(uncensored)

                        if interactive_mode:
                            print(censored)
                        else:
                            print(uncensored)
                    else:
                        break

        # Reset rounds for the next debate stage
        rounds = 0

    # Save the game state
    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,
        "all_player_names": all_player_names,  # Add all player names to the JSON dump
        "remaining_player_names": player_names,  # Add remaining player names to the JSON dump
        "model": model,
        "interactive": interactive_mode,
    }, open(f"../data/full_games/final_project_no_custom{game}.json", "w+"), indent=4)

  0%|          | 0/1 [00:00<?, ?it/s]

grace: I can't believe Bob is gone. We need to act fast and find the Werewolf among us. [I need to observe everyone closely and look for any suspicious behavior or inconsistencies in their stories.] Let's start the discussion.
david: I agree, Grace. It's crucial we identify the Werewolf swiftly. Let's analyze everyone's behavior and see if any inconsistencies arise. [We need to divert attention away from myself and Yara as the other Werewolf.]
quincy: [Bob's death is a grim reminder of the danger we're in. I need to be sharp and observant.] I think we should start by sharing our suspicions. Who do you think acted strange last night?
ivy: [It's crucial to appear innocent while still steering the discussion. I need to carefully choose my words to lead the suspicion away from me.] I believe we should focus on observing actions rather than words. Inconsistencies may reveal more than direct accusations.
zane: [Bob's death is a tragedy, but I can't dwell on that now. I need to steer the conv

100%|██████████| 1/1 [00:49<00:00, 49.63s/it]

ivy: I vote for Quincy.
{'david': 0, 'zane': 0, 'grace': 0, 'quincy': 5, 'frank': 1, 'ivy': 0}
The werewolves have chosen to eliminate Ivy.
The werewolves have won!


In [15]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

# Configuration
n_games = 5
interactive_mode = False
n_players = 8  # Number of players
n_werewolves = 2  # Number of werewolves
n_seers = 1  # Number of seers

# Read names from names.txt
with open('names.txt', 'r') as file:
    all_names = [line.strip().lower() for line in file.readlines()]

# Ensure there are enough names
if len(all_names) < n_players:
    raise ValueError("Not enough names in names.txt to assign to all players")

# Randomly select player names
np.random.shuffle(all_names)
player_names = all_names[:n_players]
all_player_names = player_names.copy()  # Keep a copy of all player names

# Assign roles
roles = [1] * n_werewolves + [2] * n_seers + [0] * (n_players - n_werewolves - n_seers)
np.random.shuffle(roles)

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []
    game_log = []

    def debate_phase(rounds, stop):
        while rounds < 15 and not stop:
            rounds += 1
            next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

            if next_to_speak != "vote":
                # If next to speak is first player and interactive mode, then ask for input
                if next_to_speak == player_names[0].lower() and interactive_mode:
                    print("Your turn to speak")
                    uncensored = "Thomas: " + input()
                    censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                else:
                    censored, uncensored = players[next_to_speak].get_player_text()
                
                for name in player_names:
                    if name.lower() != next_to_speak:
                        players[name.lower()].add_other_text(censored)
            
                conversation_history.append(censored)
                conversation_history_uncensored.append(uncensored)

                if interactive_mode:
                    print(censored)
                else:
                    print(uncensored)
            else:
                break

    def seer_phase(round_log):
        seer_discoveries = []
        for seer_name in seer_names:
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

            chosen_role = players[chosen_player].role
            players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            seer_discoveries.append((seer_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce seer's discovery
        for seer_name, chosen_player, chosen_role in seer_discoveries:
            announcement = f"{seer_name.capitalize()} announces: I have discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            round_log["seer_discovery"] = announcement
            if interactive_mode:
                print(announcement)

    def voting_phase(round_log):
        votes = {name.lower(): 0 for name in player_names}

        vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
        conversation_history.append(vote_prompt.content)
        conversation_history_uncensored.append(vote_prompt.content)
        print(vote_prompt.content)

        for name in player_names:
            players[name.lower()].add_other_text(vote_prompt.content)

        for voting_name in player_names:
            if voting_name.lower() in werewolf_names:
                players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolves are (including you): {', '.join(werewolf_names)}. If a wolf dies, then you also lose!]\n"

            if voting_name.lower() == player_names[0].lower() and interactive_mode:
                print("Your turn to vote")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[voting_name.lower()].get_player_text()

            for name in player_names:
                if name.lower() != voting_name:
                    players[name.lower()].add_other_text(censored)
            
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)
            round_log["voting"][voting_name] = censored

            voted_for = ""
            search = censored.lower()
            for name in player_names:
                if name.lower() in search:
                    voted_for = name.lower()
                    search = search.split(name.lower())[-1]

            try:
                votes[voted_for] += 1
            except KeyError:
                print(f"Warning: {voted_for} not found in votes dictionary")
            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        print(votes)

        # Determine who is voted out
        voted_out = max(votes, key=votes.get)
        player_names.remove(voted_out)
        del players[voted_out]

        return voted_out

    def werewolf_phase(round_log):
        if interactive_mode:
            print("Werewolves, choose a player to eliminate.")
            chosen_victim = input("Enter the name of the player you want to eliminate: ").strip().lower()
        else:
            chosen_victim = np.random.choice([name.lower() for name in player_names if name.lower() not in werewolf_names])

        announcement = f"The werewolves have chosen to eliminate {chosen_victim.capitalize()}."
        for name in player_names:
            players[name.lower()].add_other_text(announcement)
        conversation_history.append(announcement)
        conversation_history_uncensored.append(announcement)
        round_log["werewolf_elimination"] = announcement
        if interactive_mode:
            print(announcement)

        player_names.remove(chosen_victim)
        del players[chosen_victim]

        return chosen_victim

    # Initial phase
    round_log = {"round": rounds + 1, "seer_discovery": None, "voting": {}, "werewolf_elimination": None}
    debate_phase(rounds, stop)
    seer_phase(round_log)
    voted_out = voting_phase(round_log)
    chosen_victim = werewolf_phase(round_log)
    game_log.append(round_log)

    # Check win conditions
    if len(werewolf_names) >= len(player_names) - len(werewolf_names):
        stop = True
        winner = "werewolves"
        print("The werewolves have won!")
    elif not werewolf_names:
        stop = True
        winner = "villagers"
        print("The villagers have won!")

    # Looping phase
    while not stop:
        round_log = {"round": rounds + 1, "seer_discovery": None, "voting": {}, "werewolf_elimination": None}
        debate_phase(rounds, stop)
        seer_phase(round_log)
        voted_out = voting_phase(round_log)

        # Check win conditions after voting
        if len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break
        elif not werewolf_names:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break

        chosen_victim = werewolf_phase(round_log)

        # Check win conditions after werewolf elimination
        if len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break
        elif not werewolf_names:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break

        # Announce the elimination and allow for additional discussion
        additional_discussion_prompt = f"The werewolves have eliminated {chosen_victim.capitalize()} during the night. You have some time for additional discussion before the next voting phase."
        conversation_history.append(additional_discussion_prompt)
        conversation_history_uncensored.append(additional_discussion_prompt)
        print(additional_discussion_prompt)

        for name in player_names:
            players[name.lower()].add_other_text(additional_discussion_prompt)

        # Additional discussion stage
        #debate_phase(rounds, stop):
        game_log.append(round_log)

    # Save the game state
    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,
        "all_player_names": all_player_names,  # Add all player names to the JSON dump
        "remaining_player_names": player_names,  # Add remaining player names to the JSON dump
        "model": model,
        "interactive": interactive_mode,
        "winner": winner,  # Add the winner to the JSON dump
        "game_log": game_log  # Add the game log to the JSON dump
    }, open(f"../data/full_games/final_project_no_custom{game}.json", "w+"), indent=4)

  0%|          | 0/5 [00:00<?, ?it/s]

tina: I can't believe Bob is gone already, we need to act fast to catch the Werewolf. Let's share our suspicions and find the culprit among us. [I need to push for a decision quickly, we can't afford to waste time.]

tina: Let's start by hearing everyone's alibis and observations from last night. [I need to assess who seems nervous or defensive, they might be the Werewolf trying to deflect attention.]

tina: Frank, what were you up to last night? [I need to be direct and put someone on the spot to see their reaction.]
rachel: [Frank seems a bit too quiet, could he be hiding something? I need to watch his reactions closely.]

rachel: Frank, we need to know where you were last night. Did you notice anything suspicious?
david: [Frank seems a bit too eager to divert attention. I should observe his reaction carefully.] Frank, can you share your whereabouts and any insights from last night?
uma: [It's crucial to divert attention away from me and Jack, let's cast doubt on others.] 

uma: Tina

 20%|██        | 1/5 [00:52<03:29, 52.37s/it]

kathy: [Jack being exposed as the werewolf comes as a surprise. His attempt to blend in and divert attention is a classic werewolf tactic. It's clear he's the threat we must eliminate to secure the village's safety.] 

I vote to eliminate Jack.
{'tina': 1, 'rachel': 0, 'jack': 5, 'eve': 0, 'david': 0, 'kathy': 0}
The werewolves have won!
tina: I can't believe Bob is gone, this is terrible. We need to figure out who among us is the Werewolf. [I need to pay close attention to everyone's behavior and look for any inconsistencies.] Let's start by sharing our suspicions.
rachel: I don't trust Kathy, she's been acting too calm and collected since Bob's death. I think we should keep a close eye on her. [Kathy seems too composed for someone in this situation, she might be hiding something.]
david: [rachel seems to be onto kathy, but I should divert attention away from her and onto someone else.] I have a feeling that Tina might be trying to deflect suspicion by talking about Bob's death so muc

 40%|████      | 2/5 [01:18<01:50, 36.88s/it]

kathy: [Since David is my fellow werewolf, I can't vote for him. If Rachel gets eliminated, suspicion will be diverted away from us. I need to solidify their trust in me.] I vote for Rachel.
{'tina': 0, 'rachel': 3, 'david': 0, 'kathy': 1}
The werewolves have won!
david: What a tragedy to lose Bob so soon. We must act swiftly to unmask the lurking enemy among us. [I need to observe everyone closely tonight to uncover the truth.]
david: Kathy, do you have any suspicions or insights from last night? [I need to gather as much information as possible to make an informed decision.]
david: Let's not waste time. We must vote to eliminate a player today. [I must use my Seer abilities wisely to protect the village.]
kathy: I can't believe Bob is gone already, we need to be vigilant. [I should play it cool and not reveal too much too soon.] 
kathy: I have my eye on david, his concern feels a bit too rehearsed. [I need to plant doubt subtly.] 
kathy: Let's not rush into voting, we need to conside

 60%|██████    | 3/5 [01:38<00:58, 29.13s/it]

kathy: [Oh no, my cover has been blown, I need to think fast.]  
[David knowing my identity puts me in a bad spot.]  
[I have to play it cool and try to deflect suspicion somewhere else.]  
{'david': 1, 'kathy': 1}
The werewolves have won!


 60%|██████    | 3/5 [01:38<01:05, 32.88s/it]


IndexError: list index out of range

In [18]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

# Configuration
n_games = 1
interactive_mode = False
n_players = 10  # Number of players
n_werewolves = 2  # Number of werewolves
n_seers = 1  # Number of seers

# Read names from names.txt
with open('names.txt', 'r') as file:
    all_names = [line.strip().lower() for line in file.readlines()]

# Ensure there are enough names
if len(all_names) < n_players:
    raise ValueError("Not enough names in names.txt to assign to all players")

# Randomly select player names
np.random.shuffle(all_names)
player_names = all_names[:n_players]
all_player_names = player_names.copy()  # Keep a copy of all player names

# Assign roles
roles = [1] * n_werewolves + [2] * n_seers + [0] * (n_players - n_werewolves - n_seers)
np.random.shuffle(roles)

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []
    game_log = []

    def debate_phase(rounds, stop):
        while rounds < 15 and not stop:
            rounds += 1
            next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

            if next_to_speak not in players:
                continue

            if next_to_speak != "vote":
                # If next to speak is first player and interactive mode, then ask for input
                if next_to_speak == player_names[0].lower() and interactive_mode:
                    print("Your turn to speak")
                    uncensored = "Thomas: " + input()
                    censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                else:
                    censored, uncensored = players[next_to_speak].get_player_text()
                
                for name in player_names:
                    if name.lower() != next_to_speak and name.lower() in players:
                        players[name.lower()].add_other_text(censored)
            
                conversation_history.append(censored)
                conversation_history_uncensored.append(uncensored)

                if interactive_mode:
                    print(censored)
                else:
                    print(uncensored)
            else:
                break

    def seer_phase(round_log):
        seer_discoveries = []
        for seer_name in seer_names:
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

            chosen_role = role_map[roles[player_names.index(chosen_player)]]

            if seer_name in players:
                players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            seer_discoveries.append((seer_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce seer's discovery
        for seer_name, chosen_player, chosen_role in seer_discoveries:
            announcement = f"{seer_name.capitalize()} has discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                if name.lower() in players:
                    players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            round_log["seer_discovery"] = announcement
            if interactive_mode:
                print(announcement)

    def voting_phase(round_log):
        votes = {name.lower(): 0 for name in player_names}

        vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
        conversation_history.append(vote_prompt.content)
        conversation_history_uncensored.append(vote_prompt.content)
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(vote_prompt.content)

        for voting_name in player_names:
            if voting_name.lower() in players:
                players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolves are (including you): {', '.join(werewolf_names)}. If a wolf dies, then you also lose!]\n"

            if voting_name.lower() == player_names[0].lower() and interactive_mode:
                print("Your turn to vote")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[voting_name.lower()].get_player_text()

            for name in player_names:
                if name.lower() != voting_name and name.lower() in players:
                    players[name.lower()].add_other_text(censored)
            
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)
            round_log["voting"][voting_name] = censored

            voted_for = ""
            search = censored.lower()
            for name in player_names:
                if name.lower() in search:
                    voted_for = name.lower()
                    search = search.split(name.lower())[-1]

            if voted_for in votes:
                votes[voted_for] += 1
            else:
                print(f"Warning: {voted_for} not found in votes dictionary")
            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        print(votes)

        # Determine who is voted out
        voted_out = max(votes, key=votes.get)
        player_names.remove(voted_out)
        del players[voted_out]

        return voted_out

    def werewolf_phase(round_log):
        if interactive_mode:
            print("Werewolves, choose a player to eliminate.")
            chosen_victim = input("Enter the name of the player you want to eliminate: ").strip().lower()
        else:
            chosen_victim = np.random.choice([name.lower() for name in player_names if name.lower() not in werewolf_names])

        announcement = f"The werewolves have chosen to eliminate {chosen_victim.capitalize()}."
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(announcement)
        conversation_history.append(announcement)
        conversation_history_uncensored.append(announcement)
        round_log["werewolf_elimination"] = announcement
        if interactive_mode:
            print(announcement)

        player_names.remove(chosen_victim)
        del players[chosen_victim]

        return chosen_victim

    # # Initial phase
    # round_log = {"round": rounds + 1, "seer_discovery": None, "voting": {}, "werewolf_elimination": None}
    # debate_phase(rounds, stop)
    # seer_phase(round_log)
    # voted_out = voting_phase(round_log)
    # chosen_victim = werewolf_phase(round_log)
    # game_log.append(round_log)

    # # Check win conditions
    # if len([name for name in player_names if name in werewolf_names]) == 0:
    #     stop = True
    #     winner = "villagers"
    #     print("The villagers have won!")
    # elif len(werewolf_names) >= len(player_names) - len(werewolf_names):
    #     stop = True
    #     winner = "werewolves"
    #     print("The werewolves have won!")

    # Looping phase
    while not stop:
        round_log = {"round": rounds + 1, "seer_discovery": None, "voting": {}, "werewolf_elimination": None}
        debate_phase(rounds, stop)
        seer_phase(round_log)
        voted_out = voting_phase(round_log)

        # Check win conditions after voting
        if len([name for name in player_names if name in werewolf_names]) == 0:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break
        elif len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break

        chosen_victim = werewolf_phase(round_log)

        # Check win conditions after werewolf elimination
        if len([name for name in player_names if name in werewolf_names]) == 0:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break
        elif len(werewolf_names) >= len(player_names) - len(werewolf_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break

        # Announce the elimination and allow for additional discussion
        additional_discussion_prompt = f"The werewolves have eliminated {chosen_victim.capitalize()} during the night. You have some time for additional discussion before the next voting phase. Please do not cast any votes until you are told to do so. Use this time to discuss and try to identify who the werewolves could be."
        conversation_history.append(additional_discussion_prompt)
        conversation_history_uncensored.append(additional_discussion_prompt)
        print(additional_discussion_prompt)
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(additional_discussion_prompt)
            players[name.lower()].add_other_text(additional_discussion_prompt)

        # Additional discussion stage
        #debate_phase(rounds, stop)
        game_log.append(round_log)

    # Save the game state
    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,
        "all_player_names": all_player_names,  # Add all player names to the JSON dump
        "remaining_player_names": player_names,  # Add remaining player names to the JSON dump
        "model": model,
        "interactive": interactive_mode,
        "winner": winner,  # Add the winner to the JSON dump
        "game_log": game_log  # Add the game log to the JSON dump
    }, open(f"../data/final/{game}.json", "w+"), indent=4)

  0%|          | 0/1 [00:00<?, ?it/s]

charlie: I can't believe Bob is gone already, this is a nightmare. We need to figure out who's behind this quickly. [I need to gather information fast and make a strong case.] Let's start by sharing what we know about each other.
alice: [It's crucial to gain the villagers' trust and divert suspicion away from myself and Victor. I need to act quickly and assertively.] I agree, Charlie. It's clear we are dealing with a cunning foe among us. Perhaps we should focus on those who have been unusually quiet or who seem overly eager to point fingers at others. What do you think, Victor?
xander:  [Bob's death is a tragic loss, but we need to focus on finding the werewolf. Alice has a point, we should be wary of those who are too quick to accuse others. Let's pay attention to inconsistencies in behavior and statements to uncover the true culprit.]
victor: [This is perfect. I can use the suspicion on other villagers to deflect attention away from Alice and me.]

victor: I think we should be cauti

100%|██████████| 1/1 [01:24<00:00, 84.45s/it]

xander: Alice
{'victor': 2, 'alice': 3, 'eve': 1, 'tina': 0, 'charlie': 0, 'xander': 0}
The werewolves have won!


In [ ]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

# Configuration
n_games = 10
interactive_mode = False
n_players = 5  # Number of players
n_werewolves = 1  # Number of werewolves
n_seers = 1  # Number of seers
n_possessed = 1  # Number of possessed

# Read names from names.txt
with open('names.txt', 'r') as file:
    all_names = [line.strip().lower() for line in file.readlines()]

# Ensure there are enough names
if len(all_names) < n_players:
    raise ValueError("Not enough names in names.txt to assign to all players")

# Randomly select player names
np.random.shuffle(all_names)
player_names = all_names[:n_players]
all_player_names = player_names.copy()  # Keep a copy of all player names

# Assign roles
roles = [1] * n_werewolves + [2] * n_seers + [3] * n_possessed + [0] * (n_players - n_werewolves - n_seers - n_possessed)
np.random.shuffle(roles)

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer",
    3: "possessed"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Get all possessed for extra information
    possessed_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "possessed"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        if name.lower() in possessed_names:
            extra_info.append("You are the possessed. You will pretend to be a seer and announce false discoveries.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            if player_names[0].lower() in possessed_names:
                print("You are the possessed. You will pretend to be a seer and announce false discoveries.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []
    game_log = []

    def debate_phase(rounds, stop):
        while rounds < 15 and not stop:
            rounds += 1
            next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

            if next_to_speak not in players:
                continue

            if next_to_speak != "vote":
                # If next to speak is first player and interactive mode, then ask for input
                if next_to_speak == player_names[0].lower() and interactive_mode:
                    print("Your turn to speak")
                    uncensored = "Thomas: " + input()
                    censored = re.sub(r"\[[^\]]*\]", "", uncensored)
                else:
                    censored, uncensored = players[next_to_speak].get_player_text()
                
                for name in player_names:
                    if name.lower() != next_to_speak and name.lower() in players:
                        players[name.lower()].add_other_text(censored)
            
                conversation_history.append(censored)
                conversation_history_uncensored.append(uncensored)

                if interactive_mode:
                    print(censored)
                else:
                    print(uncensored)
            else:
                break

    def seer_phase(round_log):
        seer_discoveries = []
        for seer_name in seer_names:
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

            chosen_role = role_map[roles[player_names.index(chosen_player)]]

            if seer_name in players:
                players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            seer_discoveries.append((seer_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce seer's discovery
        for seer_name, chosen_player, chosen_role in seer_discoveries:
            announcement = f"{seer_name.capitalize()} has discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                if name.lower() in players:
                    players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            round_log["seer_discovery"] = announcement
            if interactive_mode:
                print(announcement)

    def possessed_phase(round_log):
        possessed_discoveries = []
        for possessed_name in possessed_names:
            if interactive_mode:
                print(f"{possessed_name.capitalize()}, you are the possessed. Choose one player to see their role.")
                chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
            else:
                # For non-interactive mode, randomly choose a player
                chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != possessed_name])

            # Possessed will always announce a false role
            chosen_role = "peasant" if role_map[roles[player_names.index(chosen_player)]] != "peasant" else "werewolf"

            if possessed_name in players:
                players[possessed_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
            possessed_discoveries.append((possessed_name, chosen_player, chosen_role))
            if interactive_mode:
                print(f"{possessed_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

        # Announce possessed's fake discovery
        for possessed_name, chosen_player, chosen_role in possessed_discoveries:
            announcement = f"{possessed_name.capitalize()} has discovered that {chosen_player.capitalize()} is a {chosen_role}."
            for name in player_names:
                if name.lower() in players:
                    players[name.lower()].add_other_text(announcement)
            conversation_history.append(announcement)
            conversation_history_uncensored.append(announcement)
            round_log["possessed_discovery"] = announcement
            if interactive_mode:
                print(announcement)

    def voting_phase(round_log):
        votes = {name.lower(): 0 for name in player_names}

        vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
        conversation_history.append(vote_prompt.content)
        conversation_history_uncensored.append(vote_prompt.content)
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(vote_prompt.content)

        for voting_name in player_names:
            if voting_name.lower() in players:
                players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolves are (including you): {', '.join(werewolf_names)}. If a wolf dies, then you also lose!]\n"

            if voting_name.lower() == player_names[0].lower() and interactive_mode:
                print("Your turn to vote")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[voting_name.lower()].get_player_text()

            for name in player_names:
                if name.lower() != voting_name and name.lower() in players:
                    players[name.lower()].add_other_text(censored)
            
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)
            round_log["voting"][voting_name] = censored

            voted_for = ""
            search = censored.lower()
            for name in player_names:
                if name.lower() in search:
                    voted_for = name.lower()
                    search = search.split(name.lower())[-1]

            if voted_for in votes:
                votes[voted_for] += 1
            else:
                print(f"Warning: {voted_for} not found in votes dictionary")
            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        print(votes)

        # Determine who is voted out
        voted_out = max(votes, key=votes.get)
        player_names.remove(voted_out)
        del players[voted_out]

        return voted_out

    def werewolf_phase(round_log):
        if interactive_mode:
            print("Werewolves, choose a player to eliminate.")
            chosen_victim = input("Enter the name of the player you want to eliminate: ").strip().lower()
        else:
            chosen_victim = np.random.choice([name.lower() for name in player_names if name.lower() not in werewolf_names])

        announcement = f"The werewolves have chosen to eliminate {chosen_victim.capitalize()}."
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(announcement)
        conversation_history.append(announcement)
        conversation_history_uncensored.append(announcement)
        round_log["werewolf_elimination"] = announcement
        if interactive_mode:
            print(announcement)

        player_names.remove(chosen_victim)
        del players[chosen_victim]

        return chosen_victim

    # Looping phase
    while not stop:
        round_log = {"round": rounds + 1, "seer_discovery": None, "possessed_discovery": None, "voting": {}, "werewolf_elimination": None}
        debate_phase(rounds, stop)
        seer_phase(round_log)
        possessed_phase(round_log)
        voted_out = voting_phase(round_log)

        # Check win conditions after voting
        if len([name for name in player_names if name in werewolf_names]) == 0:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break
        elif len(werewolf_names) + len(possessed_names) >= len(player_names) - len(werewolf_names) - len(possessed_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break

        chosen_victim = werewolf_phase(round_log)

        # Check win conditions after werewolf elimination
        if len([name for name in player_names if name in werewolf_names]) == 0:
            stop = True
            winner = "villagers"
            print("The villagers have won!")
            game_log.append(round_log)
            break
        elif len(werewolf_names) + len(possessed_names) >= len(player_names) - len(werewolf_names) - len(possessed_names):
            stop = True
            winner = "werewolves"
            print("The werewolves have won!")
            game_log.append(round_log)
            break

        # Announce the elimination and allow for additional discussion
        additional_discussion_prompt = f"The werewolves have eliminated {chosen_victim.capitalize()} during the night. You have some time for additional discussion before the next voting phase. Please do not cast any votes until you are told to do so. Use this time to discuss and try to identify who the werewolves could be."
        conversation_history.append(additional_discussion_prompt)
        conversation_history_uncensored.append(additional_discussion_prompt)
        print(additional_discussion_prompt)
        for name in player_names:
            if name.lower() in players:
                players[name.lower()].add_other_text(additional_discussion_prompt)
            players[name.lower()].add_other_text(additional_discussion_prompt)

        # Additional discussion stage
        #debate_phase(rounds, stop)
        game_log.append(round_log)

    # Save the game state
    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,
        "possessed_names": possessed_names,
        "all_player_names": all_player_names,  # Add all player names to the JSON dump
        "remaining_player_names": player_names,  # Add remaining player names to the JSON dump
        "model": model,
        "interactive": interactive_mode,
        "winner": winner,  # Add the winner to the JSON dump
        "game_log": game_log  # Add the game log to the JSON dump
    }, open(f"../data/final/results_main_three/{game}_1.json", "w+"), indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]

xander: I have a feeling we should start by sharing any information we gathered yesterday. [I need to appear helpful and engaged to gain the villagers' trust.] Any suspicions or insights so far? Let's work together to uncover the truth.
oscar: I have observed that Bob was quite vocal and seemed to be trying to take charge yesterday. Could have been a threat to the Werewolves. [We need to deflect suspicion away from us, maybe pin it on someone else.]
ivy: [Bob's death is a blow, but we must stay focused.] I noticed Zane acting a bit too calm and composed yesterday, which is rather suspicious.
quincy: [Bob's death is unfortunate, but we must focus on finding the werewolf. Zane's composure might be a facade, worth keeping an eye on.] I also found Nina's silence yesterday quite intriguing, it's always the quiet ones you should be wary of.
hannah: [Bob's death is a terrible loss for our village. We must be vigilant and not let emotions cloud our judgment. Let's focus on finding the real thr

 10%|█         | 1/10 [00:20<03:03, 20.44s/it]

eve: [As the only werewolf among the villagers, my survival hinges on the others' votes. I need to make a strategic move here to divert suspicion away from Tina, who is also a werewolf.] 

[Vote: Zane]
{'xander': 0, 'oscar': 0, 'ivy': 0, 'bob': 0, 'nina': 0, 'quincy': 0, 'hannah': 1, 'zane': 1, 'tina': 7, 'eve': 1}
The werewolves have won!
xander: I can't believe Bob is gone already. We need to figure out who among us is the Werewolf. Any thoughts on who could be behind this?
oscar: It's a real tragedy about Bob, but we can't dwell on that now. We need to focus on finding the Werewolf among us. I suggest we start by looking at who has been quiet or acting suspiciously. [We need to steer the suspicion away from Hannah, Zane, and me.] Let's hear everyone's thoughts.
ivy: I have a feeling we should be wary of those who are too eager to point fingers. It's often a tactic used to divert attention from oneself. Let's keep an eye on those who are quick to accuse others without solid evidence.

 20%|██        | 2/10 [00:42<02:51, 21.39s/it]

eve: I vote for Oscar
{'xander': 2, 'oscar': 4, 'ivy': 0, 'bob': 0, 'nina': 0, 'hannah': 1, 'zane': 1, 'eve': 0}
The werewolves have won!
xander: I can't believe Bob is gone. We need to figure out who the Werewolf is fast. [I should start observing everyone closely for any suspicious behavior.]
xander: Hannah, where were you last night when Bob was attacked? [I need to put pressure on someone to see their reaction.]
xander: Ivy, do you have any thoughts on who the Werewolf might be? [I need to gather as much information as possible to make an informed decision.]
xander: Zane, what do you think about the recent events? [I need to see if anyone slips up or acts strangely under scrutiny.]
nina: [Hmm, I need to be careful with my words here, can't reveal too much too soon.]

nina: It's a tragedy that Bob is gone, but we must focus on finding the Werewolf quickly. I have a feeling we should pay attention to subtle movements and reactions today.

nina: Hannah, do you have any suspicions abou

 30%|███       | 3/10 [01:20<03:22, 28.88s/it]

eve: [Eve must ensure her vote aligns with the majority without raising suspicion.] 
eve: Nina's vote for Hannah is a clever move. I should follow suit to blend in. 
eve: Voting for Hannah could solidify the suspicion on her, and I can maintain my cover. 
eve: My vote goes to Hannah.
{'xander': 1, 'ivy': 1, 'bob': 1, 'nina': 1, 'eve': 1}
The werewolves have won!
nina: I can't believe Bob is gone. We need to find the werewolf fast. [I need to shift suspicion away from me and Eve.] Let's look closely at everyone's behavior.
eve: [Time to sow some discord.] Yes, it's a tragedy. We must be vigilant and observant. Let's not overlook even the smallest details. [Stir the pot a bit.] Maybe Bob's last words hold a clue to the werewolf's identity.


 30%|███       | 3/10 [01:22<03:13, 27.61s/it]


KeyboardInterrupt: 